In [1]:
import pandas as pd
import numpy as np
import glob, sys, os, time , itertools , warnings , re , json
from bs4 import BeautifulSoup
from tqdm import tqdm
import matplotlib.pyplot as plt
import multiprocessing as mp
# from DataProcessing import 
import uszipcode , pickle 
from TempFolder.TempFolder import Temp


import plotly.figure_factory as ff
# importing all necessary libraries 
import chart_studio.plotly as py 
import plotly.graph_objs as go 
import pandas as pd 
  
# some more libraries to plot graph 
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot 
# To establish connection 
init_notebook_mode(connected = True) 


warnings.filterwarnings('ignore')

with open('nogit\\path','r') as f:
    path_to_csv = f.read()
path_to_processed_csv =path_to_csv + 'SC-csv\\'
Temp.set_path(path_to_csv +'SC-temp\\')
path_to_temp_csv = path_to_csv+'SC-working-folder\\'
colummn_names = ['Description'
                 ,'Title State/Type'
                 ,'Location'
                 ,'null'
                 ,'Auction Date'
                 ,'Actual Cash Value'
                 ,'Repair Cost'
                 ,'Odometer'
                 ,'Prim Damage'
                 ,'Sec Damage'
                 ,'Price Sold or Highest Bid']
path_to_makes = glob.glob(path_to_csv+'Cars\\*')
search = uszipcode.SearchEngine(simple_zipcode=True)
path_to_all_years = list(itertools.chain.from_iterable([glob.glob(path+'\\*') for path in path_to_makes]))
search = uszipcode.SearchEngine(simple_zipcode=True)

#CPI Data
path_to_cpi = path_to_csv+'US-CPI\\SeriesReport-20191118124049_549fff.xlsx'
cpi_df = pd.read_excel(path_to_cpi,skiprows=11,index_col=0)['Annual']#[:-2]

In [2]:
print('CPI data:' , cpi_df.to_string())

CPI data: Year
1990    130.700
1991    136.200
1992    140.300
1993    144.500
1994    148.200
1995    152.400
1996    156.900
1997    160.500
1998    163.000
1999    166.600
2000    172.200
2001    177.100
2002    179.900
2003    184.000
2004    188.900
2005    195.300
2006    201.600
2007    207.342
2008    215.303
2009    214.537
2010    218.056
2011    224.939
2012    229.594
2013    232.957
2014    236.736
2015    237.017
2016    240.007
2017    245.120
2018    251.107
2019    255.657


In [3]:
# #%%time
# '''
# Parse the HTML files
# '''

# #
# #Read and parse scraped .html files 
# def process_files(path_to_years__):
#     res_df = pd.DataFrame()
#     #for each make/year folder get a list of all .html files
#     #for path_to__year in tqdm(path_to_years__):
#     for path_to__year in path_to_years__:
#         try:
#             #list of all html
#             path_to_make_years = glob.glob(path_to__year+'/*')

#             #for each .html in the folder
#             for file_path in path_to_make_years:
#             #file_path = path_to_make_years[0]
            
#                 #read file and load to the BS object
#                 with open(file_path, 'r') as f:
#                     fle = f.read()
#                 soup = BeautifulSoup(fle)

#                 rows = []
#                 line_ind = 0 
                
#                 #filter all div elements (div table)
#                 for div in soup.find_all('div',['row']):
#                     row = div.text.strip()
#                     #print(row)
#                     try:
#                         line_dict = {}
#                         i = 0
                        
#                         #clean each element and map to respected column
#                         for line in row.replace('\t','').replace('\xa0','').split('\n'):
#                             line_dict[colummn_names[i]]= line.replace('Location:','').replace('Title State/Type:','').strip()
#                             i+=1
#                         rows.append(line_dict)
#                     except Exception as e:
#                         #print(line_ind,'-',e)
#                         pass
#                     finally:
#                         line_ind+=1

#                 page_df = pd.DataFrame(rows).iloc[3:,:-1]
#                 page_df_ext = page_df.join(page_df['Description'].str.split(' ',expand=True).rename(columns={0:'Year',1:'Make',2:'Model',3:'Model 2',4:'Model 3'}))
#                 page_df_ext = page_df_ext.join(page_df_ext['Location'].str.split('-',expand=True).rename(columns={0:'State',1:'City'}))
#                 page_df_ext['City'] = page_df_ext['City'].str.strip().str.capitalize()
#                 page_df_ext['State'] = page_df_ext['State'].str.strip()
#                 res_df = res_df.append(page_df_ext,ignore_index=True,sort=False)
#                 del rows , page_df_ext , page_df
#         except:
#             continue
#     return res_df


# if __name__ == '__main__':
#     print('# files to concat' , len(path_to_all_years))
#     NCPU = mp.cpu_count()
#     print('Creating pool with ', NCPU,' CPUs')
#     pool = mp.Pool(processes=4)
#     # if not os.path.exists(path_to_processed_csv):
#     #     print('Creating output folder')
#     #     os.makedirs(path_to_processed_csv)
    
#     by_chunk_path_to_all_years = np.array_split(path_to_all_years,len(path_to_all_years) // NCPU )
#     start_ = 0
#     end_ = len(by_chunk_path_to_all_years)
#     for i in range(start_,end_):
#         print('Starting with chunk: ',i,' of ',end_)
#         path_to_all_years_split = np.array_split(by_chunk_path_to_all_years[i],NCPU)
#         ress = pool.map(process_files,path_to_all_years_split)
#         ress_df = pd.concat(ress)
#         print('Chunk ',i,' is done.')
#         ress_df.to_csv(path_to_processed_csv+'sc_'+str(i)+'.csv',index=False)
# #         i+=1
# #         if i ==2:
# #             break
#     pool.close()
#     pool.join()
#     print('all work is done')
    

In [4]:
#Read parsed files
outputs = glob.glob(path_to_processed_csv+'*')
out_l = []
errs = []
for output in tqdm(outputs):
    try:
#     print(output)
        out_l.append(pd.read_csv(output))
    except:
        errs.append(output)
out_df = pd.concat(out_l,ignore_index=True,sort=False)

if len(errs) >0:
    print('Errors:', errs)

100%|██████████| 321/321 [00:05<00:00, 63.12it/s]
Errors: ['c:\\data\\Datasets\\SC-csv\\sc_0.csv', 'c:\\data\\Datasets\\SC-csv\\sc_1.csv', 'c:\\data\\Datasets\\SC-csv\\sc_2.csv', 'c:\\data\\Datasets\\SC-csv\\sc_3.csv', 'c:\\data\\Datasets\\SC-csv\\sc_4.csv', 'c:\\data\\Datasets\\SC-csv\\sc_5.csv', 'c:\\data\\Datasets\\SC-csv\\sc_6.csv', 'c:\\data\\Datasets\\SC-csv\\sc_7.csv']


In [5]:
print(out_df.columns)

row_size = out_df.shape[0]

print('Raw dataset size: ',row_size)


out_df.head(50)


Index(['Actual Cash Value', 'Auction Date', 'Description', 'Location',
       'Odometer', 'Price Sold or Highest Bid', 'Prim Damage', 'Repair Cost',
       'Sec Damage', 'Title State/Type', 'Year', 'Make', 'Model', 'Model 2',
       'State', 'City', 'Model 3', '2', '5'],
      dtype='object')
Raw dataset size:  1048104


,Actual Cash Value,Auction Date,Description,Location,Odometer,Price Sold or Highest Bid,Prim Damage,Repair Cost,Sec Damage,Title State/Type,Year,Make,Model,Model 2,State,City,Model 3,2,5
0,"$ 5,649 USD",10/02/2019,1992 Acura INTEGRA GS,KY - LEXINGTON WEST,146073 A,"$ 3,750 USD",Front end,NaN,NaN,FL RB,1992,Acura,INTEGRA,GS,KY,Lexington west,NaN,NaN,NaN
1,NaN,06/18/2019,1992 Acura INTEGRA LS,CA - SAN JOSE,182250 E,$ 300 USD,Minor dent/scratches,NaN,NaN,CA CQ,1992,Acura,INTEGRA,LS,CA,San jose,NaN,NaN,NaN
2,NaN,05/24/2019,1992 Acura VIGOR GS,NJ - SOMERVILLE,188301 A,$ 575 USD,Front end,NaN,Mechanical,NJ CT,1992,Acura,VIGOR,GS,NJ,Somerville,NaN,NaN,NaN
3,$ 725 USD,11/26/2018,1992 Acura INTEGRA GS,WA - NORTH SEATTLE,300930 E,$ 325 USD,Front end,NaN,Mechanical,WA CT,1992,Acura,INTEGRA,GS,WA,North seattle,NaN,NaN,NaN
4,"$ 1,450 USD",09/12/2018,1992 Acura INTEGRA LS,NY - LONG ISLAND,260216 E,$ 300 USD,Front end,NaN,NaN,NY CT,1992,Acura,INTEGRA,LS,NY,Long island,NaN,NaN,NaN
5,"$ 3,250 USD",12/20/2017,1992 Acura VIGOR GS,CA - MARTINEZ,205852 E,$ 175 USD,Minor dent/scratches,NaN,Side,CA DQ,1992,Acura,VIGOR,GS,CA,Martinez,NaN,NaN,NaN
6,NaN,12/08/2016,1992 Acura LEGEND L,MN - MINNEAPOLIS,144195 E,$ 125 USD,MECHANICAL,NaN,HAIL,MN CT,1992,Acura,LEGEND,L,MN,Minneapolis,NaN,NaN,NaN
7,"$ 1,960 USD",09/13/2016,1992 Acura INTEGRA LS,TN - KNOXVILLE,309149 E,$ 975 USD,SIDE,NaN,NaN,TN CT,1992,Acura,INTEGRA,LS,TN,Knoxville,NaN,NaN,NaN
8,NaN,07/25/2016,1992 Acura LEGEND LS,MD - BALTIMORE,104173 A,$ 150 USD,MINOR DENT/SCRATCHES,NaN,NORMAL WEAR,MD CT,1992,Acura,LEGEND,LS,MD,Baltimore,NaN,NaN,NaN
9,NaN,07/19/2016,1992 Acura LEGEND L,NJ - GLASSBORO EAST,131385 A,$ 200 USD,NORMAL WEAR,NaN,NaN,NJ CT,1992,Acura,LEGEND,L,NJ,Glassboro east,NaN,NaN,NaN


In [6]:
#clean up wrong values 
out_df = out_df.loc[~out_df['Actual Cash Value'].str.contains('Odometer').fillna(False)]
out_df = out_df.loc[~out_df['Year'].str.contains('Make').fillna(False)]
out_df = out_df.loc[~out_df['Year'].str.contains('Model:').fillna(False)]
out_df = out_df.loc[~out_df['Auction Date'].str.contains('Repair Cost').fillna(False)]


#filter out canadian cars
out_df = out_df.loc[~out_df['Price Sold or Highest Bid'].str.contains('CA').fillna(False)
                               ]

out_df['Auction Date'] = pd.to_datetime(out_df['Auction Date'])

out_df['Auction_Year'] = out_df['Auction Date'].dt.year

In [7]:
#check datset size
cleaned_size = out_df.shape[0]
print('Cleaned dataset size:',cleaned_size
      ,'\nRows removed: ', row_size - cleaned_size)


Cleaned dataset size: 1046013 
Rows removed:  2091


In [8]:
out_df.columns

Index(['Actual Cash Value', 'Auction Date', 'Description', 'Location',
       'Odometer', 'Price Sold or Highest Bid', 'Prim Damage', 'Repair Cost',
       'Sec Damage', 'Title State/Type', 'Year', 'Make', 'Model', 'Model 2',
       'State', 'City', 'Model 3', '2', '5', 'Auction_Year'],
      dtype='object')

In [9]:
# Combining Prim Damages 
raw_prim_cat = out_df['Prim Damage'].str.lower().unique()
print('Cat count:',raw_prim_cat.shape[0],'\n',raw_prim_cat)

#prim damage map
prim_damage_map = {
 'burn - engine': 'burn'
, 'partial/incomplete r' : 'partial repair'
, 'frame damage reporte' : 'frame damage'

,'unknown' : 'no data'


, np.nan : 'no data'
,'price sold or highest bid' :'no data'
,'rr' : 'no data'

}
#apply mapping
out_df['Prim Damage'] = out_df['Prim Damage'].str.lower().replace(prim_damage_map)

new_prim_cat = out_df['Prim Damage'].str.lower().unique()
print('Cat count:',new_prim_cat.shape[0],'\n',new_prim_cat)

out_df['Prim Damage']= out_df['Prim Damage'].str.capitalize()

Cat count: 33 
 ['front end' 'minor dent/scratches' 'mechanical' 'side' 'normal wear'
 'hail' 'minor dents/scratche' 'stripped' 'all over' 'damage history'
 'partial/incomplete r' 'rear end' 'vandalism' 'water/flood' 'burn'
 'previous repair' 'partial repair' 'top/roof' 'rollover'
 'burn - interior' 'undercarriage' 'biohazard/chemical' 'burn - engine'
 'frame damage' 'missing/altered vin' 'unknown' 'frame damage reporte'
 'replaced vin' 'rejected repair' 'biohazardous/chemica' nan
 'cash for clunkers' 'rr']
Cat count: 28 
 ['front end' 'minor dent/scratches' 'mechanical' 'side' 'normal wear'
 'hail' 'minor dents/scratche' 'stripped' 'all over' 'damage history'
 'partial repair' 'rear end' 'vandalism' 'water/flood' 'burn'
 'previous repair' 'top/roof' 'rollover' 'burn - interior' 'undercarriage'
 'biohazard/chemical' 'frame damage' 'missing/altered vin' 'no data'
 'replaced vin' 'rejected repair' 'biohazardous/chemica'
 'cash for clunkers']


In [10]:
# out_df = out_df.join(out_df['Location'].str.split('-',expand=True).rename(columns={0:'State',1:'City'}))

In [11]:
#fix numerical values 

def alter_sting(x):    
    try:
        return float(re.sub('[\$,USD,\, ,E,A,N]','',x))
    except:
#         print(x)
        return np.nan



out_df['Actual Cash Value'] = (out_df['Actual Cash Value']
                               #.fillna(-1)
                               .map(alter_sting))
out_df['Repair Cost'] = (out_df['Repair Cost']
#                          .fillna(-1)
                         .map(alter_sting))

out_df['Price Sold or Highest Bid'] = (out_df['Price Sold or Highest Bid']
                                       #.fillna(-1)
                                       .map(alter_sting))
out_df['Year'] = (out_df['Year']
#                   .fillna(-1)
                  .map(float))

out_df = out_df.rename(columns={'Year':'Model_Year'
                               })
                      

out_df['City'] = out_df['City'].str.strip().str.capitalize()
out_df['State'] = out_df['State'].str.strip().str.upper()
out_df['Model_short'] = out_df['Model'].astype('str')
out_df['Model'] = out_df['Model'].astype('str') + ' '+out_df['Model 2'].astype('str').fillna('') + ' ' + out_df['Model 3'].fillna('').astype('str')
out_df['Make'] = out_df['Make'].str.strip().str.upper()
out_df['Model'] = out_df['Model'].str.strip().str.upper()
out_df['Model_short'] = out_df['Model_short'].str.strip().str.upper()

#rename variables - add _ where needed

out_df = out_df.rename({'Price Sold or Highest Bid':'Price_Sold_or_Highest_Bid'
                            ,'Repair Cost':'Repair_Cost'
                            ,'Actual Cash Value':'Actual_Cash_Value'
                            ,'Prim Damage':'Prim_Damage'
                            ,'Sec Damage':'Sec_Damage'
                            ,'Title State/Type':'Title_State_Type'
                            ,'Auction Date':'Auction_Date'
                            },axis=1)




out_df['Model_Year'] = out_df['Model_Year'].astype(int)

#remove unused columns 
out_df = out_df.drop(columns=['Model 2','Model 3', '2', '5'])


print('New columns names:',out_df.columns)

New columns names: Index(['Actual_Cash_Value', 'Auction_Date', 'Description', 'Location',
       'Odometer', 'Price_Sold_or_Highest_Bid', 'Prim_Damage', 'Repair_Cost',
       'Sec_Damage', 'Title_State_Type', 'Model_Year', 'Make', 'Model',
       'State', 'City', 'Auction_Year', 'Model_short'],
      dtype='object')


In [12]:
print(out_df.shape)
out_df.head()

(1046013, 17)


,Actual_Cash_Value,Auction_Date,Description,Location,Odometer,Price_Sold_or_Highest_Bid,Prim_Damage,Repair_Cost,Sec_Damage,Title_State_Type,Model_Year,Make,Model,State,City,Auction_Year,Model_short
0,5649.0,2019-10-02,1992 Acura INTEGRA GS,KY - LEXINGTON WEST,146073 A,3750.0,Front end,NaN,NaN,FL RB,1992,ACURA,INTEGRA GS,KY,Lexington west,2019,INTEGRA
1,NaN,2019-06-18,1992 Acura INTEGRA LS,CA - SAN JOSE,182250 E,300.0,Minor dent/scratches,NaN,NaN,CA CQ,1992,ACURA,INTEGRA LS,CA,San jose,2019,INTEGRA
2,NaN,2019-05-24,1992 Acura VIGOR GS,NJ - SOMERVILLE,188301 A,575.0,Front end,NaN,Mechanical,NJ CT,1992,ACURA,VIGOR GS,NJ,Somerville,2019,VIGOR
3,725.0,2018-11-26,1992 Acura INTEGRA GS,WA - NORTH SEATTLE,300930 E,325.0,Front end,NaN,Mechanical,WA CT,1992,ACURA,INTEGRA GS,WA,North seattle,2018,INTEGRA
4,1450.0,2018-09-12,1992 Acura INTEGRA LS,NY - LONG ISLAND,260216 E,300.0,Front end,NaN,NaN,NY CT,1992,ACURA,INTEGRA LS,NY,Long island,2018,INTEGRA


In [13]:
#format odometer data

#for null model - replace all E  & N values into "No Data" (np.nan)

def odometer_null(x):
    if 'E' in x or 'N' in x:
        return np.nan
    else:
        return alter_sting(x)


    
#create a dictionary for max E value for each model 

odometr_dict = (out_df.loc[out_df['Odometer'].str.contains('A')]
                 .assign(odometer_num = lambda x: x['Odometer'].map(alter_sting)
                        )
                 .groupby('Description')['odometer_num']
                 .max()
                #  [['Odometer','odometer_num']]
                )

#replace E and N values into max of A category
def odometer_replace(x):
    if 'E' in x['Odometer'] or 'N' in x['Odometer']:
        try:
            return odometr_dict[x['Description']]
        except:
            return np.nan
    else:
        return alter_sting(x['Odometer'])


#create null model
out_df = out_df.assign(Odometer_Null = lambda x: x['Odometer'].map(odometer_null))

#create replace model:

out_df = out_df.assign(Odometer_Replace = lambda x: x.apply(odometer_replace,axis=1))

#create odometer original
out_df = out_df.rename(columns={'Odometer':'Odometer_Original'}).assign(Odometer_Original = lambda x: x['Odometer_Original'].map(alter_sting) )

# create Model_Age
# current_year = 2019
out_df['Model_Age'] = (out_df['Auction_Year'] - out_df['Model_Year'] ) + 1 #.dt.days // 365 
# out_df['Model_Year'] = out_df['Model_Year'].dt.year

In [15]:
print(out_df.shape)
out_df.tail()

(1046013, 20)


,Actual_Cash_Value,Auction_Date,Description,Location,Odometer_Original,Price_Sold_or_Highest_Bid,Prim_Damage,Repair_Cost,Sec_Damage,Title_State_Type,Model_Year,Make,Model,State,City,Auction_Year,Model_short,Odometer_Null,Odometer_Replace,Model_Age
1048099,21521.0,2016-08-15,2016 Nissan ROGUE S/SL,FL - JACKSONVILLE EAST,0.0,4900.0,Front end,20642.00,BIOHAZARD/CHEMICAL,FL CD,2016,NISSAN,ROGUE S/SL,NaN,NaN,2016,ROGUE,NaN,49819.0,1
1048100,NaN,2016-08-12,2016 Nissan ALTIMA 2.5,AZ - TUCSON,1939.0,6900.0,Front end,NaN,NaN,AZ SC,2016,NISSAN,ALTIMA 2.5,NaN,NaN,2016,ALTIMA,1939.0,1939.0,1
1048101,21208.0,2016-08-12,2016 Nissan ROGUE S/SL,TX - HOUSTON,11051.0,2500.0,All over,26661.32,UNDERCARRIAGE,TX SV,2016,NISSAN,ROGUE S/SL,NaN,NaN,2016,ROGUE,11051.0,11051.0,1
1048102,18340.0,2016-08-12,2016 Nissan SENTRA S/S,IL - CHICAGO SOUTH,2607.0,2500.0,Front end,22830.00,UNDERCARRIAGE,IL SC,2016,NISSAN,SENTRA S/S,NaN,NaN,2016,SENTRA,NaN,79448.0,1
1048103,18000.0,2016-08-11,2016 Nissan SENTRA S/S,PA - PITTSBURGH NORTH,1742.0,4900.0,Front end,NaN,NaN,PA SC,2016,NISSAN,SENTRA S/S,NaN,NaN,2016,SENTRA,1742.0,1742.0,1


In [15]:
#lookup county name from City - State

def find_county(x):
    try:
        return search.by_city_and_state(x[1] , x[0])[0].county
    except:
        return 'None'


locs = out_df['Location'].unique()
locs = locs[~pd.isnull(locs)]

state_city = [l[:2] for l in  list(map(lambda x: x.split(' - '),   locs ) ) #if len(l)>1
             ]





county = list(map(find_county,   state_city ) )

county = [', '.join([l.strip() for l in (county[i].replace('County','')+', '+state_city[i][0]).split(',') ])
 for i in range(len(county))]

loc_lookup = pd.DataFrame({'Location':locs,'County':county}).set_index('Location')['County']

loc_lookup


def map_countries(x):
    try:
        return loc_lookup.loc[x]
    except:
        return 'None'
        
out_df['County'] = out_df['Location'].map(map_countries)
out_df['make_key'] = out_df.assign(Model_Year = lambda x: x['Model_Year'].astype(str))[['Make','Model_short','Model_Year']].agg('_'.join,axis=1)


print(out_df.shape)
out_df.head(10)

(1046013, 22)


,Actual_Cash_Value,Auction_Date,Description,Location,Odometer,Price_Sold_or_Highest_Bid,Prim_Damage,Repair_Cost,Sec_Damage,Title_State_Type,...,Model,State,City,Auction_Year,Model_short,Odometer_Null,Odometer_Replace,Model_Age,County,make_key
0,5649.0,2019-10-02,1992 Acura INTEGRA GS,KY - LEXINGTON WEST,146073 A,3750.0,Front end,NaN,NaN,FL RB,...,INTEGRA GS,KY,Lexington west,2019,INTEGRA,146073.0,146073.0,28,"Fayette, KY",ACURA_INTEGRA_1992
1,NaN,2019-06-18,1992 Acura INTEGRA LS,CA - SAN JOSE,182250 E,300.0,Minor dent/scratches,NaN,NaN,CA CQ,...,INTEGRA LS,CA,San jose,2019,INTEGRA,NaN,178757.0,28,"Santa Clara, CA",ACURA_INTEGRA_1992
2,NaN,2019-05-24,1992 Acura VIGOR GS,NJ - SOMERVILLE,188301 A,575.0,Front end,NaN,Mechanical,NJ CT,...,VIGOR GS,NJ,Somerville,2019,VIGOR,188301.0,188301.0,28,"Somerset, NJ",ACURA_VIGOR_1992
3,725.0,2018-11-26,1992 Acura INTEGRA GS,WA - NORTH SEATTLE,300930 E,325.0,Front end,NaN,Mechanical,WA CT,...,INTEGRA GS,WA,North seattle,2018,INTEGRA,NaN,197187.0,27,"King, WA",ACURA_INTEGRA_1992
4,1450.0,2018-09-12,1992 Acura INTEGRA LS,NY - LONG ISLAND,260216 E,300.0,Front end,NaN,NaN,NY CT,...,INTEGRA LS,NY,Long island,2018,INTEGRA,NaN,178757.0,27,"Queens, NY",ACURA_INTEGRA_1992
5,3250.0,2017-12-20,1992 Acura VIGOR GS,CA - MARTINEZ,205852 E,175.0,Minor dent/scratches,NaN,Side,CA DQ,...,VIGOR GS,CA,Martinez,2017,VIGOR,NaN,216614.0,26,"Contra Costa, CA",ACURA_VIGOR_1992
6,NaN,2016-12-08,1992 Acura LEGEND L,MN - MINNEAPOLIS,144195 E,125.0,Mechanical,NaN,HAIL,MN CT,...,LEGEND L,MN,Minneapolis,2016,LEGEND,NaN,233596.0,25,"Hennepin, MN",ACURA_LEGEND_1992
7,1960.0,2016-09-13,1992 Acura INTEGRA LS,TN - KNOXVILLE,309149 E,975.0,Side,NaN,NaN,TN CT,...,INTEGRA LS,TN,Knoxville,2016,INTEGRA,NaN,178757.0,25,"Knox, TN",ACURA_INTEGRA_1992
8,NaN,2016-07-25,1992 Acura LEGEND LS,MD - BALTIMORE,104173 A,150.0,Minor dent/scratches,NaN,NORMAL WEAR,MD CT,...,LEGEND LS,MD,Baltimore,2016,LEGEND,104173.0,104173.0,25,"Baltimore city, MD",ACURA_LEGEND_1992
9,NaN,2016-07-19,1992 Acura LEGEND L,NJ - GLASSBORO EAST,131385 A,200.0,Normal wear,NaN,NaN,NJ CT,...,LEGEND L,NJ,Glassboro east,2016,LEGEND,131385.0,131385.0,25,"Gloucester, NJ",ACURA_LEGEND_1992


In [16]:
# cpi adjustment currency fields
base_year = 2010
def cpi_adj_Actual_Cash_Value(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['Auction_Year']]) * x['Actual_Cash_Value'],2)
    except ValueError:
        return x['Actual_Cash_Value']

print(out_df.shape)
out_df['Actual_Cash_Value_adj'] = out_df.apply(cpi_adj_Actual_Cash_Value
                                            ,axis=1)



def cpi_adj_Price_Sold_or_Highest_Bid(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['Auction_Year']]) * x['Price_Sold_or_Highest_Bid'],2)
    except ValueError:
        return x['Price_Sold_or_Highest_Bid']

print(out_df.shape)
out_df['Price_Sold_or_Highest_Bid_adj'] = out_df.apply(cpi_adj_Price_Sold_or_Highest_Bid
                                            ,axis=1)

def cpi_adj_Repair_Cost(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['Auction_Year']]) * x['Repair_Cost'],2)
    except ValueError:
        return x['Repair_Cost']

print(out_df.shape)
out_df['Repair_Cost_adj'] = out_df.apply(cpi_adj_Repair_Cost
                                            ,axis=1)


#create a foregin key for match
out_df['make_key'] = out_df.assign(Model_Year = lambda x: x['Model_Year'].astype(str))[['Make','Model_short','Model_Year']].agg('_'.join,axis=1)

out_df = pd.merge(out_df,pd.read_csv(path_to_csv +'Us-zipcodes\\fips_codes.csv',index_col=0), on='County',how='left')

out_df = (out_df.drop('State',axis=1)
                    .join(out_df['County'].str.split(', ',expand=True)[1]
                    .rename('State')))

print(out_df.shape)

(1046013, 22)
(1046013, 23)
(1046013, 24)
(1046013, 26)


In [17]:
#remove outlyer data
out_df['is_outlyer'] = 0 
#mark repair cost oult
out_df.loc[out_df.Repair_Cost_adj >=out_df.Repair_Cost_adj.quantile(0.99),'is_outlyer'] = 1
# out_df.loc[out_df.Repair_Cost_adj >=out_df.Repair_Cost_adj.quantile(0.99),'Repair_Cost_adj'] = np.nan

#Actual cash value
out_df.loc[out_df.Actual_Cash_Value_adj >=out_df.Actual_Cash_Value_adj.quantile(0.99),'is_outlyer'] = 1
# out_df.loc[out_df.Actual_Cash_Value_adj >=out_df.Actual_Cash_Value_adj.quantile(0.99),'Actual_Cash_Value'] = np.nan

#price sold
out_df.loc[out_df.Price_Sold_or_Highest_Bid_adj >=out_df.Price_Sold_or_Highest_Bid_adj.quantile(0.99),'is_outlyer'] = 1
# out_df.loc[out_df.Price_Sold_or_Highest_Bid_adj >=out_df.Price_Sold_or_Highest_Bid_adj.quantile(0.99),'Actual_Cash_Value'] = np.nan

#odometer replace
out_df.loc[out_df.Odometer_Replace >=out_df.Odometer_Replace.quantile(0.99),'is_outlyer'] = 1
# out_df.loc[out_df.Odometer_Replace >=out_df.Odometer_Replace.quantile(0.99),'Actual_Cash_Value'] = np.nan

#odometer replace
out_df.loc[out_df.Odometer_Replace >=out_df.Odometer_Null.quantile(0.99),'is_outlyer'] = 1
# out_df.loc[out_df.Odometer_Replace >=out_df.Odometer_Null.quantile(0.99),'Actual_Cash_Value'] = np.nan


#
print('outlyers detected: ',out_df['is_outlyer'].sum() )

print(out_df.shape)

outlyers detected:  199640
(1046013, 27)


In [18]:
out_df.columns

Index(['Actual_Cash_Value', 'Auction_Date', 'Description', 'Location',
       'Odometer_Origin', 'Price_Sold_or_Highest_Bid', 'Prim_Damage',
       'Repair_Cost', 'Sec_Damage', 'Title_State_Type', 'Model_Year', 'Make',
       'Model', 'City', 'Auction_Year', 'Model_short', 'Odometer_Null',
       'Odometer_Replace', 'Model_Age', 'County', 'make_key',
       'Actual_Cash_Value_adj', 'Price_Sold_or_Highest_Bid_adj',
       'Repair_Cost_adj', 'FIPS', 'State', 'is_outlyer'],
      dtype='object')

In [19]:
Temp.save_obj(out_df, 'out_df_working')
out_df.to_csv(path_to_temp_csv+'full_working_df.csv')
# out_df = Temp.load_obj('out_df_working')
print(out_df.shape)

Object <2772120324104> saved to: c:\data\Datasets\SC-temp\
(1046013, 27)


In [20]:
'''
END OF MAIN DATASET PREPROCESSING
'''

'\nEND OF MAIN DATASET PREPROCESSING\n'

In [21]:
'''
JOINING WITH DEMOGRAPHIC DATA 
'''


#read the demography data 
path_to_demograpyh =path_to_csv+'SC-working-folder\\demography.csv'

demog = pd.read_csv(path_to_demograpyh)

print('Demographic features:\n' , demog.columns.tolist())



Demographic features:
 ['index', 'Unemp-Pct-2014', 'Unemp-Pct-2015', 'Unemp-Pct-2016', 'Unemp-Pct-2017', 'Unemp-Pct-2018', 'Unemp-Pct-2013', 'Unemp-Pct-2012', 'Unemp-Pct-2011', 'Unemp-Pct-2010', 'Income_adj-1990', 'Income_adj-1991', 'Income_adj-1992', 'Income_adj-1993', 'Income_adj-1994', 'Income_adj-1995', 'Income_adj-1996', 'Income_adj-1997', 'Income_adj-1998', 'Income_adj-1999', 'Income_adj-2000', 'Income_adj-2001', 'Income_adj-2002', 'Income_adj-2003', 'Income_adj-2004', 'Income_adj-2005', 'Income_adj-2006', 'Income_adj-2007', 'Income_adj-2008', 'Income_adj-2009', 'Income_adj-2010', 'Income_adj-2011', 'Income_adj-2012', 'Income_adj-2013', 'Income_adj-2014', 'Income_adj-2015', 'Income_adj-2016', 'Income_adj-2017', 'CENSUS2010', 'CENSUS2011', 'CENSUS2012', 'CENSUS2013', 'CENSUS2014', 'CENSUS2015', 'CENSUS2016', 'CENSUS2017', 'CENSUS2018', 'GRNDTOT_2009', 'GRNDTOT_2010', 'GRNDTOT_2011', 'GRNDTOT_2012', 'GRNDTOT_2013', 'GRNDTOT_2014', 'GRNDTOT_2016']


In [22]:

#read carfax data
path_to_carfax =path_to_csv+'SC-carfax\\processed\\MSRP.csv'

carfax = pd.read_csv(path_to_carfax,index_col=0) 




#read carfax data
path_to_carfax =path_to_csv+'SC-carfax\\processed\\MSRP.csv'

carfax = pd.read_csv(path_to_carfax,index_col=0) 
carfax['make'] = carfax['make'].str.upper()
carfax['model'] = carfax['model'].str.upper()
carfax['year'] = carfax['year'].astype(str)


#create a foreging key
carfax['make_key'] = carfax[['make','model','year']].agg('_'.join,axis=1)


carfax['Original_MSRP_mean'] =carfax[['Original_MSRP_low','Original_MSRP_high']].mean(axis=1)


#CPI adjusted income 
carfax['year'] = carfax['year'].astype(int)
def cpi_adj_Original_MSRP_mean(x):
    try:
        
        return round((cpi_df[base_year] / cpi_df[x['year']]) * x['Original_MSRP_mean'],2)
    except KeyError:
        return x['Original_MSRP_mean']


carfax['Original_MSRP_mean_adj'] = carfax.apply(cpi_adj_Original_MSRP_mean
                                            ,axis=1)


print(carfax.shape)
carfax.head()


(4144, 8)


,make,model,year,Original_MSRP_low,Original_MSRP_high,Original_MSRP_mean,Original_MSRP_mean_adj,make_key
0,ACURA,CL,2001,28460.0,30810.0,29635.0,36488.37,ACURA_CL_2001
1,ACURA,CL,2002,28530.0,32880.0,30705.0,37217.40,ACURA_CL_2002
2,ACURA,CL,2003,28700.0,31050.0,29875.0,35404.47,ACURA_CL_2003
3,ACURA,ILX,2013,25900.0,34400.0,30150.0,28221.47,ACURA_ILX_2013
4,ACURA,ILX,2014,26900.0,34600.0,30750.0,28323.63,ACURA_ILX_2014


In [23]:
# join original and carfex
out_df_carfix = pd.merge(out_df, carfax,  how='left', on ='make_key')#.drop(columns='make_key')
print(out_df_carfix.shape)
out_df_carfix.head()

(1046013, 34)


,Actual_Cash_Value,Auction_Date,Description,Location,Odometer_Origin,Price_Sold_or_Highest_Bid,Prim_Damage,Repair_Cost,Sec_Damage,Title_State_Type,...,FIPS,State,is_outlyer,make,model,year,Original_MSRP_low,Original_MSRP_high,Original_MSRP_mean,Original_MSRP_mean_adj
0,5649.0,2019-10-02,1992 Acura INTEGRA GS,KY - LEXINGTON WEST,146073 A,3750.0,Front end,NaN,NaN,FL RB,...,NaN,KY,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2019-06-18,1992 Acura INTEGRA LS,CA - SAN JOSE,182250 E,300.0,Minor dent/scratches,NaN,NaN,CA CQ,...,6085.0,CA,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2019-05-24,1992 Acura VIGOR GS,NJ - SOMERVILLE,188301 A,575.0,Front end,NaN,Mechanical,NJ CT,...,34035.0,NJ,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,725.0,2018-11-26,1992 Acura INTEGRA GS,WA - NORTH SEATTLE,300930 E,325.0,Front end,NaN,Mechanical,WA CT,...,53033.0,WA,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1450.0,2018-09-12,1992 Acura INTEGRA LS,NY - LONG ISLAND,260216 E,300.0,Front end,NaN,NaN,NY CT,...,NaN,NY,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
out_df_carfix.loc[out_df_carfix['Make'] == 'TOYOTA','Model_short'].unique()

array(['FJ40', 'LAND', 'FJ', 'CORONA', 'COROLLA', 'PICK-UP', 'LANDCRUISR',
       'LANDCRUISE', 'HILUX', '4D', 'OTHER', 'CELICA', 'PICK', 'PICKUP',
       'STANDARD', 'SUPRA', 'SR5', 'CRESSIDA', 'DELUXE', 'DOLPHIN',
       'STARLET', 'TERCEL', 'VAN', 'CAMRY', 'MR2', 'FORKLIFT', '4RUNNER',
       'LONG', 'CAB', 'SEABREEZE', 'EXTRA', 'PREVIA', 'SKID', 'PASEO',
       'SHORT', 'TRUCK', 'T100', 'COROLLA/DX', 'CELICA/ST', 'CAMRY/DX',
       'AVALON', 'PREVIA/DX', 'TACOMA', 'RAV4', '4RUNNER/SR', 'RAV', '4',
       'SIENNA', 'SIENNA/CE', 'ECHO', 'SOLARA', 'ARISTO', 'TUNDRA',
       'TUNDRA/SR5', 'HIGHLANDER', 'PRIUS', 'SEQUOIA', '1/2',
       'COROLLA/S/', 'MATRIX', 'MATRIX/XR', 'CORROLA', 'COROLLA/MA',
       'CORROLLA', '2DR', 'XB', 'HIGLANDER', 'SIENNA/LE', 'YARIS',
       'CAMRY-LE', 'AV', 'ALL', 'COROLL', 'CAM', 'SCION', 'VENZA',
       'COROLLA/LE', 'CAMRY/SE/L', 'COR', 'CAMRY/LE/X', 'AVALON/LIM',
       'FR-S', 'HIGHLNDER', 'MIRAI', 'IA', '86', 'C-HR'], dtype=object)

In [25]:
#join car data with demographic data
car_demo_joined = (pd.merge(out_df_carfix, demog,left_on='County',right_on='index',how='left')
                    .drop(columns=['Location','Description','Sec_Damage','Title_State_Type', 'City','make', 'model','year'])					
                    )
				


print(car_demo_joined.shape, car_demo_joined.head(),'\n\n' ,car_demo_joined.columns)

(1046013, 80)    Actual_Cash_Value Auction_Date Odometer_Origin  Price_Sold_or_Highest_Bid  \
0             5649.0   2019-10-02        146073 A                     3750.0   
1                NaN   2019-06-18        182250 E                      300.0   
2                NaN   2019-05-24        188301 A                      575.0   
3              725.0   2018-11-26        300930 E                      325.0   
4             1450.0   2018-09-12        260216 E                      300.0   

            Prim_Damage  Repair_Cost  Model_Year   Make       Model  \
0             Front end          NaN        1992  ACURA  INTEGRA GS   
1  Minor dent/scratches          NaN        1992  ACURA  INTEGRA LS   
2             Front end          NaN        1992  ACURA    VIGOR GS   
3             Front end          NaN        1992  ACURA  INTEGRA GS   
4             Front end          NaN        1992  ACURA  INTEGRA LS   

   Auction_Year  ... CENSUS2016  CENSUS2017  CENSUS2018  GRNDTOT_2009  \
0    

In [26]:
#create master demographic field 

# 1 Unemp-Pct

unemp_columns  = [col for col in car_demo_joined.columns if 'Unemp-Pct' in col]
print('working with ' ,unemp_columns)
def fetch_unemp_null(x):
    ''' Return the actual on the action date unemployment, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'Unemp-Pct-{auction_y}']
    except:
        return

car_demo_joined['Unemp_pct_null'] = car_demo_joined.apply(fetch_unemp_null, axis=1)

unemp_available = np.array([2014,2015,2016,2017,2018,2013,2012,2011,2010])
def fetch_unemp_closest(x):
    ''' Return the actual on the action date unemployment, if none, return abs closest'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'Unemp-Pct-{auction_y}']
    except:
        distance = unemp_available - x['Auction_Year']
        closest_year_ix = np.array([el if el<=0 else -99999 for el in distance]).argmax()
        closest_year = unemp_available[closest_year_ix]
        return x[f'Unemp-Pct-{closest_year}']

car_demo_joined['Unemp_pct_closest'] = car_demo_joined.apply(fetch_unemp_closest, axis=1)


working with  ['Unemp-Pct-2014', 'Unemp-Pct-2015', 'Unemp-Pct-2016', 'Unemp-Pct-2017', 'Unemp-Pct-2018', 'Unemp-Pct-2013', 'Unemp-Pct-2012', 'Unemp-Pct-2011', 'Unemp-Pct-2010']


In [27]:
# 2 Income_adj
inc_columns  = [col for col in car_demo_joined.columns if 'Income_adj' in col]
print('working with ' ,inc_columns)

def fetch_inc_null(x):
    ''' Return the actual on the action date income, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'Income_adj-{auction_y}']
    except:
        return

car_demo_joined['Income_adj_null'] = car_demo_joined.apply(fetch_inc_null, axis=1)


income_available = np.arange(1990,2018)
def fetch_inc_closest(x):
    ''' Return the actual on the action date income, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'Income_adj-{auction_y}']
    except:
        distance = income_available - x['Auction_Year']
        closest_year_ix = np.array([el if el<=0 else -99999 for el in distance]).argmax()
        closest_year = income_available[closest_year_ix]
        return x[f'Income_adj-{closest_year}']

car_demo_joined['Income_adj_closest'] = car_demo_joined.apply(fetch_inc_closest, axis=1)


working with  ['Income_adj-1990', 'Income_adj-1991', 'Income_adj-1992', 'Income_adj-1993', 'Income_adj-1994', 'Income_adj-1995', 'Income_adj-1996', 'Income_adj-1997', 'Income_adj-1998', 'Income_adj-1999', 'Income_adj-2000', 'Income_adj-2001', 'Income_adj-2002', 'Income_adj-2003', 'Income_adj-2004', 'Income_adj-2005', 'Income_adj-2006', 'Income_adj-2007', 'Income_adj-2008', 'Income_adj-2009', 'Income_adj-2010', 'Income_adj-2011', 'Income_adj-2012', 'Income_adj-2013', 'Income_adj-2014', 'Income_adj-2015', 'Income_adj-2016', 'Income_adj-2017']


In [28]:
# 3 CENSUS
CENSUS_columns  = [col for col in car_demo_joined.columns if 'CENSUS' in col]
print('working with ' ,CENSUS_columns)

def CENSUS_null(x):
    ''' Return the actual on the action date CENSUS, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'CENSUS{auction_y}']
    except:
        return

car_demo_joined['CENSUS_null'] = car_demo_joined.apply(CENSUS_null, axis=1)


CENSUS_available = np.arange(2010,2019)
def CENSUS_closest(x):
    ''' Return the actual on the action date CENSUS, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'CENSUS{auction_y}']
    except:
        distance = CENSUS_available - x['Auction_Year']
        closest_year_ix = np.array([el if el<=0 else -99999 for el in distance]).argmax()
        
        closest_year = CENSUS_available[closest_year_ix]
        return x[f'CENSUS{closest_year}']

car_demo_joined['CENSUS_closest'] = car_demo_joined.apply(CENSUS_closest, axis=1)


working with  ['CENSUS2010', 'CENSUS2011', 'CENSUS2012', 'CENSUS2013', 'CENSUS2014', 'CENSUS2015', 'CENSUS2016', 'CENSUS2017', 'CENSUS2018']


In [29]:
# 4 - GRNDTOT
GRNDTOT_columns  = [col for col in car_demo_joined.columns if 'GRNDTOT' in col]
print('working with ' ,GRNDTOT_columns)

def GRNDTOT_null(x):
    ''' Return the actual on the action date GRNDTOT, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'GRNDTOT_{auction_y}']
    except:
        return

car_demo_joined['GRNDTOT_null'] = car_demo_joined.apply(GRNDTOT_null, axis=1)


GRNDTOT_available = np.array([2009,2010,2011,2012,2013,2014,2016])
def GRNDTOT_closest(x):
    ''' Return the actual on the action date GRNDTOT, if none, return None'''
    try:
        auction_y = str(x['Auction_Year'])
        return x[f'GRNDTOT_{auction_y}']
    except:

        distance = GRNDTOT_available - x['Auction_Year']
        closest_year_ix = np.array([el if el<=0 else -99999 for el in distance]).argmax()
        
        closest_year = GRNDTOT_available[closest_year_ix]
        return x[f'GRNDTOT_{closest_year}']

car_demo_joined['GRNDTOT_closest'] = car_demo_joined.apply(GRNDTOT_closest, axis=1)


working with  ['GRNDTOT_2009', 'GRNDTOT_2010', 'GRNDTOT_2011', 'GRNDTOT_2012', 'GRNDTOT_2013', 'GRNDTOT_2014', 'GRNDTOT_2016']


In [30]:
# # 5 - CPOPARST
# CPOPARST_columns  = [col for col in car_demo_joined.columns if 'CPOPARST' in col]
# print('working with ' ,GRNDTOT_columns)

# def CPOPARST_null(x):
#     ''' Return the actual on the action date CPOPARST, if none, return None'''
#     try:
#         auction_y = str(x['Auction_Year'])
#         return x[f'CPOPARST_{auction_y}']
#     except:
#         return

# car_demo_joined['CPOPARST_null'] = car_demo_joined.apply(CPOPARST_null, axis=1)


# CPOPARST_available = np.array([2009,2010,2011,2012,2013,2014,2016])
# def CPOPARST_closest(x):
#     ''' Return the actual on the action date CPOPARST, if none, return None'''
#     try:
#         auction_y = str(x['Auction_Year'])
#         return x[f'CPOPARST_{auction_y}']
#     except:
#         closest_year_ix = np.abs( CPOPARST_available - x['Auction_Year']).argmin()


#         closest_year = CPOPARST_available[closest_year_ix]
#         return x[f'CPOPARST_{closest_year}']

# car_demo_joined['CPOPARST_closest'] = car_demo_joined.apply(CPOPARST_closest, axis=1)


In [31]:
#remove yearly demographic data 
col_to_rem = [unemp_columns,inc_columns ,CENSUS_columns,GRNDTOT_columns
#,CPOPARST_columns
]
from itertools import chain
col_to_rem_fl = list(chain.from_iterable(col_to_rem))
#col_to_rem#[np.array(i) for i in col_to_rem]
car_demo_joined['State'] = car_demo_joined['County'].str.split(', ',expand=True)[1]

car_demo_joined = car_demo_joined.drop(columns=col_to_rem_fl)

In [32]:
#create indicator variables
car_demo_joined['Odometer_is_damaged'] = np.where(car_demo_joined['Odometer_Null'].isna(),1,0)

In [33]:
car_demo_joined.loc[2].to_dict()

{'Actual_Cash_Value': nan,
 'Auction_Date': Timestamp('2019-05-24 00:00:00'),
 'Odometer_Origin': '188301 A',
 'Price_Sold_or_Highest_Bid': 575.0,
 'Prim_Damage': 'Front end',
 'Repair_Cost': nan,
 'Model_Year': 1992,
 'Make': 'ACURA',
 'Model': 'VIGOR GS',
 'Auction_Year': 2019,
 'Model_short': 'VIGOR',
 'Odometer_Null': 188301.0,
 'Odometer_Replace': 188301.0,
 'Model_Age': 28,
 'County': 'Somerset, NJ',
 'make_key': 'ACURA_VIGOR_1992',
 'Actual_Cash_Value_adj': nan,
 'Price_Sold_or_Highest_Bid_adj': 490.43,
 'Repair_Cost_adj': nan,
 'FIPS': 34035.0,
 'State': 'NJ',
 'is_outlyer': 0,
 'Original_MSRP_low': nan,
 'Original_MSRP_high': nan,
 'Original_MSRP_mean': nan,
 'Original_MSRP_mean_adj': nan,
 'index': 'Somerset, NJ',
 'Unemp_pct_null': nan,
 'Unemp_pct_closest': 3.4,
 'Income_adj_null': nan,
 'Income_adj_closest': 85888.01684073107,
 'CENSUS_null': nan,
 'CENSUS_closest': 331164.0,
 'GRNDTOT_null': nan,
 'GRNDTOT_closest': 6848.0,
 'Odometer_is_damaged': 0}

In [34]:
car_demo_joined.columns

Index(['Actual_Cash_Value', 'Auction_Date', 'Odometer_Origin',
       'Price_Sold_or_Highest_Bid', 'Prim_Damage', 'Repair_Cost', 'Model_Year',
       'Make', 'Model', 'Auction_Year', 'Model_short', 'Odometer_Null',
       'Odometer_Replace', 'Model_Age', 'County', 'make_key',
       'Actual_Cash_Value_adj', 'Price_Sold_or_Highest_Bid_adj',
       'Repair_Cost_adj', 'FIPS', 'State', 'is_outlyer', 'Original_MSRP_low',
       'Original_MSRP_high', 'Original_MSRP_mean', 'Original_MSRP_mean_adj',
       'index', 'Unemp_pct_null', 'Unemp_pct_closest', 'Income_adj_null',
       'Income_adj_closest', 'CENSUS_null', 'CENSUS_closest', 'GRNDTOT_null',
       'GRNDTOT_closest', 'Odometer_is_damaged'],
      dtype='object')

In [35]:
#save output
car_demo_joined.to_csv(path_to_temp_csv+'joint_working_df.csv')
Temp.save_obj(car_demo_joined,'car_demo_joined')

Object <2771906386568> saved to: c:\data\Datasets\SC-temp\


In [37]:
'''
ANALYSIS
'''

'\nANALYSIS\n'

In [ ]:
# del out_df
out_df = Temp.load_obj('out_df_working')
# car_demo_joined = Temp.load_obj('car_demo_joined')

In [ ]:
out_df.groupby('Make')['Model_Year'].count().sort_values(ascending=False)[:60]

In [ ]:
# filterd_acv = out_df[var_to_plot]
# filterd_acv_round = filterd_acv


# filterd_acv_round_vc = filterd_acv_round.value_counts().to_frame().reset_index().rename({'index':'level' , var_to_plot:'count'}, axis=1)
# filterd_acv_round_vc.to_csv('csv/outly.csv', index=False)

In [ ]:
#histogram

s = 20
e = 80
#display modes of the distribution 
try:
    filterd_acv = out_df[var_to_plot].loc[(out_df[var_to_plot] >s )& (out_df[var_to_plot] <e)]

    print('Counts:\n#>100k:'
                ,len(out_df[var_to_plot].loc[out_df[var_to_plot]>=1e5]))

except:
    filterd_acv = out_df[var_to_plot]




n = 1
filterd_acv_round = (filterd_acv // n ) * n

filterd_acv.plot.hist(title= var_to_plot+f' (between {s} and {e})',color='k', alpha=.4)

filterd_acv_round_vc = filterd_acv_round.value_counts().to_frame().reset_index().rename({'index':'level' , var_to_plot:'count'}, axis=1)
filterd_acv_round_vc.to_csv('csv/outly.csv', index=False)
# filterd_acv_round_vc

In [ ]:


#plot a histogram of observed cash_values
fig= plt.figure()
ax = filterd_acv.plot.hist()
ax.set(title = 'Histogram for Actual_Cash_Value < 10k');





In [ ]:
#fit  normat distribution

from scipy import stats

x = np.linspace(0,filterd_acv.max(),len(filterd_acv))
mu , std = stats.norm.fit(filterd_acv.values)

ax = filterd_acv.plot.hist(normed=True)
plt.plot(x, stats.norm.pdf(x , mu, std))
ax.set(title = 'Histogram for Actual_Cash_Value < 10k');
print('K-S p-value:', stats.kstest(filterd_acv.values , 'norm')[1])



In [ ]:

out_df['Model_Age'].describe().astype(int)

In [ ]:
#model_age
#check model age statsВ

out_df.loc[out_df['Model_Age'].loc[out_df['Model_Age']<0].index]

In [ ]:

numerical_fields =  ['Actual_Cash_Value'
               ,'Auction_Year'
       #'Odometer'
       , 'Price_Sold_or_Highest_Bid'
       , 'Repair_Cost'
        ,'Odometer_Null','Odometer_Replace'
        ,'Model_Age'
        ]

categorical_variables = ['Prim_Damage','Make','Model'
]


In [ ]:
#categories for all categorical variables 

# categorial_variables = ['Prim_Damage'
#                         ,'Sec_Damage'
#                         ,'Make'
#                         ,'Model' # - to many categories >2000
#
                        #  ]  
out_df[categorical_variables]  =out_df[categorical_variables].astype('category')

print(out_df[categorical_variables].describe().to_string(),'\n\n')

cat_df = {}
for col in categorical_variables:
    print(col, ' - ', out_df[col].cat.categories.tolist(),'\n')
    cat_df[col] = out_df[col].cat.categories.tolist()


In [ ]:
model_year_count = out_df['Model_Year'].value_counts()
print('total count:',out_df.shape[0],'\ntotal count (model_year >= 2000):', out_df.loc[out_df.Model_Year >=2000].shape[0])
model_year_count.head(20)Z

In [ ]:
out_df.columns

In [ ]:
numerical_fields_bins = {'Actual_Cash_Value' : [],
'Price_Sold_or_Highest_Bid': [],
'Repair_Cost': [],
'Odometer_Null': [],
'Odometer_Replace': [],
'Model_Age': }

In [ ]:
#histomgram for conitnuous variables
fig , axs = plt.subplots(len(numerical_fields) ,2, figsize=(12,29),sharex=False)


i=0
for var in numerical_fields:
    # hist_df_lb ,hist_df_hb = out_df[var].loc[lambda x: x <=3e4] , out_df[var].loc[lambda x: x >=1e4]
    df_to_print = (out_df[var] // 100).value_counts().to_frame().sort_values(by=var)
    df_to_print.to_csv(f'csv/{var}_hist.csv')
    print(df_to_print)
    # hist_df_lb.plot.hist(bins=10, ax= axs[i][0])
    # hist_df_hb.plot.hist(bins=10, ax= axs[i][1])
    
    # axs[i][0].set(title = var + ' <100k bin')
    # axs[i][1].set(title = var + ' >100k bin')
    i+=1

In [ ]:
out_df.columns

In [ ]:
#value distribution for Description 

desc_val_dist = out_df['Description'].value_counts()

desc_val_dist[:30]

In [ ]:
#value distribution for Prim Damage 

prim_val_dist = out_df['Prim_Damage'].value_counts()

prim_val_dist[:30]

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(cond_on_make)

In [ ]:
plt.figure()
ax = by_couny[:20].plot.bar()
ax.set(title='Top 20 Counties by # records',ylabel='# observations');

In [ ]:
#categories for all categorical variables 
# categorial_variables = ['Prim_Damage'
#                         ,'Sec_Damage'
#                         ,'Make'
#                         ,'Model' # - to many categories >2000
#
                        #  ]  
out_df[categorial_variables]  =out_df[categorial_variables].astype('category')

print(out_df[categorial_variables].describe().to_string(),'\n\n')

cat_df = {}
for col in categorial_variables:
    print(col, ' - ', out_df[col].cat.categories.tolist(),'\n')
    cat_df[col] = out_df[col].cat.categories.tolist()


In [ ]:
print('end')
